In [1]:
!pip install groq
!pip install -qU langchain_community
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you h

In [2]:
import json
from groq import Groq
from langchain_community.document_loaders import PDFPlumberLoader

In [3]:
import os

os.environ["GROQ_API_KEY"] = "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"



In [4]:
system_prompt = """You are an AI assistant designed to extract structured resume data.
                   Always respond with a strictly valid JSON object. Use `null` for missing values,
                   ensuring compliance with JSON standards. Do not include explanations,
                   comments, or any additional text outside the JSON structure.
                """



In [5]:
human_prompt = """
             **Task:** Extract key information from the following resume text.

            **Resume Text:**
            {context}

            **Instructions:**
            Please extract the following information and format it in a clear structure:

            1. **Contact Information:**
            - Name:
            - Email:
            - Phone Number:
            - Website/Portfolio/LinkedIn:
            - Github Profile:

            2. **Education:**
            - Institution Name:
            - Degree:
            - Graduation Date:

            3. **Experience:**
            - Job Title:
            - Company Name:
            - Location:
            - Dates of Employment:
            - Description:

            5. **Skills:**
            - Skills:

            **Question:**
            Extract this information as a structured and valid JSON object. Use `null` for missing or unavailable valuesDo not include explanations,
                   comments, or any additional text outside the JSON structure.
        """


In [6]:
# Initialize Groq client once
client = Groq()

# Your system and human prompts (define these before the loop)
system_prompt = system_prompt
human_prompt = human_prompt

def extract_text_pdf(pdf_path):
    loader = PDFPlumberLoader(pdf_path)
    docs = loader.load()
    text = ''
    for doc in docs:
        text += doc.page_content
    return text

def process_resume(pdf_path, index):
    # Extract text from PDF
    context = extract_text_pdf(pdf_path)
    
    # Call LLM API
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": human_prompt.format(context=context)
            }
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    # Process the streamed output
    output = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        print(content, end="")
        output += content
    
    # Extract JSON content
    json_start = output.find('{')
    json_end = output.rfind('}') + 1
    json_content = output[json_start:json_end]
    
    # Save JSON to file
    try:
        json_data = json.loads(json_content)
        output_filename = f"resume_data_pdf_{index}.json"
        with open(output_filename, 'w') as f:
            json.dump(json_data, f, indent=2)
        print(f"\nJSON saved successfully to {output_filename}")
        return True
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON for {pdf_path}: {e}")
        # Save raw output for debugging
        with open(f"raw_output_pdf_{index}.json", 'w') as f:
            f.write(output)
        return False

In [7]:
pdf_path1= "/kaggle/input/appprseinput/18448085.rank1.pdf"
pdf_path2= "/kaggle/input/appprseinput/83816738.rank2.pdf"
pdf_path3= "/kaggle/input/appprseinput/22946204.rank3.pdf"
pdf_path4= "/kaggle/input/appprseinput/50328713.rank4.pdf"
pdf_path5= "/kaggle/input/appprseinput/17823436.rank5.pdf"

In [8]:
# Process each PDF separately
pdf_paths = [pdf_path1,pdf_path2,pdf_path3,pdf_path4,pdf_path5]  # Replace with your actual PDF paths
for i, pdf_path in enumerate(pdf_paths, start=1):
    print(f"\nProcessing resume {i}: {pdf_path}")
    process_resume(pdf_path, i)
    print(f"Completed processing {pdf_path}")


Processing resume 1: /kaggle/input/appprseinput/18448085.rank1.pdf
{
"Contact Information": {
"Name": null,
"Email": null,
"PhoneNumber": null,
"Website/Portfolio/LinkedIn": null,
"Github Profile": null
},
"Education": [
{
"Institution Name": "Masters of Science: Industrial and Systems Engineering",
"Degree": "Industrial and Systems Engineering",
"Graduation Date": "Aug 2016"
},
{
"Institution Name": "Osmania University",
"Degree": "Mechanical Engineering",
"Graduation Date": "May 2014"
}
],
"Experience": [
{
"Job Title": "Data Analyst",
"CompanyName": "Company Name ï¼​ State",
"Location": null,
"Dates of Employment": "07/2016 to Current",
"Description": null
},
{
"Job Title": "Manager",
"CompanyName": "Company Name ï¼​ State",
"Location": null,
"Dates of Employment": "09/2015 to 05/2016",
"Description": "Undertook a leadership and advisory role in training newcomers to hone their culinary and behavioral skills."
},
{
"Job Title": "Intern",
"CompanyName": "Company Name",
"Location": n